<a href="https://colab.research.google.com/github/ThisIsFarhan/pytorch-codes/blob/main/08_ANN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Workflow
"""
1. Dataloader objects
2. Training Loop
3. Evalatuation
"""

'\n1. Dataloader objects\n2. Training Loop\n3. Evalatuation\n'

In [5]:
!pip install kagglehub[pandas-datasets]

In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

In [7]:
torch.manual_seed(42)

In [10]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [8]:
import kagglehub
from kagglehub import KaggleDatasetAdapter

file_path = "fashion-mnist_train.csv"

df = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,
  "zalando-research/fashionmnist",
  file_path
)



In [9]:
df.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,0,0,0,0,0,0,0,5,0,...,0,0,0,30,43,0,0,0,0,0
3,0,0,0,0,1,2,0,0,0,0,...,3,0,0,0,0,1,0,0,0,0
4,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
X = df.iloc[:,1:].values
y = df.iloc[:,0].values

In [12]:
type(X)

numpy.ndarray

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [14]:
X_train = X_train/255.0
X_test = X_test/255.0

In [15]:
X_train

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.00392157, 0.        ,
        0.        ]])

In [16]:
class CustomDataset(Dataset):
  def __init__(self, features, labels):
    self.features = torch.tensor(features, dtype=torch.float32)
    self.labels = torch.tensor(labels, dtype=torch.long)

  def __len__(self):
    return self.features.shape[0]

  def __getitem__(self, index):
    return self.features[index], self.labels[index]



In [17]:
train_dataset = CustomDataset(X_train, y_train)
test_dataset = CustomDataset(X_test, y_test)

In [18]:
len(train_dataset)

48000

In [19]:
train_dataset[0]

(tensor([0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.2275,
         0.5333, 0.0000, 0.0

In [20]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [21]:
class myNN(nn.Module):
  def __init__(self, num_features):
    super().__init__()
    self.network = nn.Sequential(
        nn.Linear(num_features, 128),
        nn.ReLU(),
        nn.Linear(128,64),
        nn.ReLU(),
        nn.Linear(64,10),
    )

  def forward(self, x):
    return self.network(x)


In [24]:
epochs = 100
lr = 0.1
model = myNN(X_train.shape[1])
model = model.to(device)
loss_function = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(),lr)

In [25]:
len(train_loader)

1500

In [26]:
for epoch in range(epochs):
  total_epoch_loss = 0
  for batch_features, batch_labels in train_loader:
    batch_features = batch_features.to(device)
    batch_labels = batch_labels.to(device)
    #forward pass
    outputs = model(batch_features)
    #calulcate loss
    loss = loss_function(outputs, batch_labels)
    #back prop
    optimizer.zero_grad()
    loss.backward()
    #update params
    optimizer.step()


    total_epoch_loss = total_epoch_loss + loss.item()

  avg_loss = total_epoch_loss/len(train_loader)
  print(f'Epoch: {epoch + 1} , Loss: {avg_loss}')

Epoch: 1 , Loss: 0.6442541582981746
Epoch: 2 , Loss: 0.432057913094759
Epoch: 3 , Loss: 0.3867000223423044
Epoch: 4 , Loss: 0.3572725569754839
Epoch: 5 , Loss: 0.3385145670423905
Epoch: 6 , Loss: 0.3232740025669336
Epoch: 7 , Loss: 0.30780292422821126
Epoch: 8 , Loss: 0.29756631531069677
Epoch: 9 , Loss: 0.28432619310791296
Epoch: 10 , Loss: 0.274511347360909
Epoch: 11 , Loss: 0.26553403203065196
Epoch: 12 , Loss: 0.2601559265678128
Epoch: 13 , Loss: 0.2529807674959302
Epoch: 14 , Loss: 0.24344130228521924
Epoch: 15 , Loss: 0.23930770195275544
Epoch: 16 , Loss: 0.2330618276372552
Epoch: 17 , Loss: 0.22821492086847622
Epoch: 18 , Loss: 0.22220238779857754
Epoch: 19 , Loss: 0.217659245159477
Epoch: 20 , Loss: 0.21119479821901768
Epoch: 21 , Loss: 0.20388950807166598
Epoch: 22 , Loss: 0.2011286676240464
Epoch: 23 , Loss: 0.19966202997385213
Epoch: 24 , Loss: 0.19421586274666092
Epoch: 25 , Loss: 0.1880985551327467
Epoch: 26 , Loss: 0.18550730106048285
Epoch: 27 , Loss: 0.18129825512506068

In [27]:
model.eval()

myNN(
  (network): Sequential(
    (0): Linear(in_features=784, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=10, bias=True)
  )
)

In [28]:
total = 0
correct = 0

with torch.no_grad():
  for batch_features, batch_labels in test_loader:
    batch_features = batch_features.to(device)
    batch_labels = batch_labels.to(device)

    output = model(batch_features)
    print("Output shape:",output.shape)
    _, predicted = torch.max(output, 1)
    print("Predicted shape:",predicted.shape)
    print("Batch_label shape:",batch_labels.shape)
    print("-"*50)
    total += batch_labels.shape[0]
    correct += (predicted == batch_labels).sum().item()

print(correct/total)


Output shape: torch.Size([32, 10])
Predicted shape: torch.Size([32])
Batch_label shape: torch.Size([32])
--------------------------------------------------
Output shape: torch.Size([32, 10])
Predicted shape: torch.Size([32])
Batch_label shape: torch.Size([32])
--------------------------------------------------
Output shape: torch.Size([32, 10])
Predicted shape: torch.Size([32])
Batch_label shape: torch.Size([32])
--------------------------------------------------
Output shape: torch.Size([32, 10])
Predicted shape: torch.Size([32])
Batch_label shape: torch.Size([32])
--------------------------------------------------
Output shape: torch.Size([32, 10])
Predicted shape: torch.Size([32])
Batch_label shape: torch.Size([32])
--------------------------------------------------
Output shape: torch.Size([32, 10])
Predicted shape: torch.Size([32])
Batch_label shape: torch.Size([32])
--------------------------------------------------
Output shape: torch.Size([32, 10])
Predicted shape: torch.Size([